# Hyperparams And Distributions

This page introduces the hyperparams, and distributions in Neuraxle. You can find [Hyperparams Distribution API here](https://www.neuraxle.org/stable/api/neuraxle.hyperparams.distributions.html), and 
[Hyperparameter Samples API here](https://www.neuraxle.org/stable/api/neuraxle.hyperparams.space.html).

Hyperparameter is a parameter drawn from a prior distribution. In Neuraxle, we have a few built-in distributions, and we are also compatible with scipy distributions. 

Create a [Uniform Distribution](https://www.neuraxle.org/stable/api/neuraxle.hyperparams.distributions.html#neuraxle.hyperparams.distributions.Uniform):

In [ ]:
from neuraxle.hyperparams.distributions import Uniform

hd = Uniform(
    min_included=-10, 
    max_included=10, 
    null_default_value=0
)

Sample the random variable using [rvs](https://www.neuraxle.org/stable/api/neuraxle.hyperparams.distributions.html#neuraxle.hyperparams.distributions.HyperparameterDistribution.rvs):

In [ ]:
sample = hd.rvs()
print(sample)

Nullify the random variable using [nullify](https://www.neuraxle.org/stable/api/neuraxle.hyperparams.distributions.html#neuraxle.hyperparams.distributions.HyperparameterDistribution.nullify):

In [ ]:
nullified_sample = hd.nullify()
assert nullified_sample == hd.null_default_value

Get the probability distribution function value at `x` using [pdf](https://www.neuraxle.org/stable/api/neuraxle.hyperparams.distributions.html#neuraxle.hyperparams.distributions.HyperparameterDistribution.pdf):

In [ ]:
pdf = hd.pdf(1)
print('pdf: {}'.format(pdf))

Get the cumulative probability distribution function value at `x` using [cdf](https://www.neuraxle.org/stable/api/neuraxle.hyperparams.distributions.html#neuraxle.hyperparams.distributions.HyperparameterDistribution.cdf)

In [ ]:
cdf = hd.cdf(1)
print('cdf: {}'.format(cdf))

## Setting And Updating Hyperparams


In Neuraxle, each step has hyperparams of type [HyperparameterSamples](https://www.neuraxle.org/stable/api/neuraxle.hyperparams.space.html#neuraxle.hyperparams.space.HyperparameterSamples), and spaces of type [HyperparameterSpace](https://www.neuraxle.org/stable/api/neuraxle.hyperparams.distributions.html#neuraxle.hyperparams.distributions.HyperparameterDistribution).  

Consider a simple pipeline that contains 2 MultiplyByN steps, and one PCA component inside a nested pipeline:

In [ ]:
from sklearn.decomposition import PCA

from neuraxle.hyperparams.distributions import RandInt
from neuraxle.hyperparams.space import HyperparameterSpace
from neuraxle.pipeline import Pipeline
from neuraxle.steps.numpy import MultiplyByN

p = Pipeline([
    ('step1', MultiplyByN(2)),
    ('step2', MultiplyByN(2)),
    Pipeline([
        PCA(n_components=4)
    ])
])

We can set or update the hyperparams, and spaces by doing the following:  

In [ ]:
p.set_hyperparams(HyperparameterSamples({
    'step1__multiply_by': 42,
    'step2__multiply_by': -10,
    'Pipeline__PCA__n_components': 2
}))

p.update_hyperparams_space(HyperparameterSamples({
    'Pipeline__PCA__n_components': 3
}))

p.set_hyperparams_space(HyperparameterSpace({
    'step1__multiply_by': RandInt(42, 50),
    'step2__multiply_by': RandInt(-10, 0),
    'Pipeline__PCA__n_components': RandInt(2, 3)
}))

p.update_hyperparams_space(HyperparameterSpace({
    'Pipeline__PCA__n_components': RandInt(2, 5)
}))

We can sample the space of random variables:

In [ ]:
samples = p.get_hyperparams_space().rvs()

assert 42 <= samples['step1__multiply_by'] <= 50
assert -10 <= samples['step2__multiply_by'] <= 0
assert samples['Pipeline__PCA__n_components'] in [2, 3]

We can get all hyperparams:

In [ ]:
samples = p.get_hyperparams()

assert 42 <= samples['step1__multiply_by'] <= 50
assert -10 <= samples['step2__multiply_by'] <= 0
assert samples['Pipeline__PCA__n_components'] in [2, 3]
assert p['Pipeline']['PCA'].get_wrapped_sklearn_predictor().n_components in [2, 3]

## Neuraxle Custom Distributions

## Scipy Distributions 

Neuraxle is compaptible with scipy distributions.
To use scipy distributions in Neuraxle, we can simply wrap them with ScipyDistributionWrapper: 

In [ ]:
from neuraxle.hyperparams.scipy_distributions import ScipyDistributionWrapper

hd = ScipyDistributionWrapper(
    scipy_distribution=randint(low=min_included, high=max_included),
    null_default_value=null_default_value
)

All of the scipy distribution methods are available:

In [ ]:
samples = get_many_samples_for(hd)

for s in samples:
    assert type(s) == int
samples_mean = np.abs(np.mean(samples))
assert samples_mean < 1.0
assert min(samples) >= -10.0
assert max(samples) <= 10.0

assert hd.pdf(-11) == 0.
assert abs(hd.pdf(-10) - 1 / (10 + 10 + 1)) < 1e-6
assert abs(hd.pdf(0) - 1 / (10 + 10 + 1)) < 1e-6
assert hd.pdf(0.5) == 0.
assert abs(hd.pdf(10) - 1 / (10 + 10 + 1)) < 1e-6
assert hd.pdf(11) == 0.

assert hd.cdf(-10.1) == 0.
assert abs(hd.cdf(-10) - 1 / (10 + 10 + 1)) < 1e-6
assert abs(hd.cdf(0) - (0 + 10 + 1) / (10 + 10 + 1)) < 1e-6
assert abs(hd.cdf(5) - (5 + 10 + 1) / (10 + 10 + 1)) < 1e-6
assert abs(hd.cdf(10) - 1.) < 1e-6
assert hd.cdf(10.1) == 1.

assert hd.logpdf(5) == -13.418938533204672
assert hd.logcdf(5) == -0.6931477538632531
assert hd.sf(5) == 0.5000002866515718
assert hd.logsf(5) == -0.693146607256966
assert np.all(hd.ppf([0.0, 0.01, 0.05, 0.1, 1 - 0.10, 1 - 0.05, 1 - 0.01, 1.0], 10))
assert hd.isf(q=0.5) == 8.798228093189323
assert hd.moment(2) == 50.50000000091249
assert hd.stats()[0]
assert hd.stats()[1]
assert np.array_equal(hd.entropy(), np.array(0.7094692666023363))
assert hd.median()
assert hd.mean() == 5.398942280397029
assert hd.std() == 4.620759921685374
assert hd.var() == 21.35142225385382
assert hd.expect() == 0.39894228040143276
interval = hd.interval(alpha=[0.25, 0.50])
assert np.all(interval[0])
assert np.all(interval[1])
assert hd.support() == (0, 10)

## SKLearn Hyperparams

SKLearnWrapper wraps sklearn predictors so that they can be compatible with Neuraxle. When you set the hyperparams of an SKLearnWrapper, it automatically sets the params of the sklearn predictor for you: 

In [ ]:
from neuraxle.hyperparams.distributions import Choice
from neuraxle.hyperparams.distributions import RandIn
from neuraxle.hyperparams.space import HyperparameterSpace
from neuraxle.steps.sklearn import SKLearnWrapper
from sklearn.tree import DecisionTreeClassifier


decision_tree_classifier = SKLearnWrapper(
    DecisionTreeClassifier(), 
    HyperparameterSpace({
        'criterion': Choice(['gini', 'entropy']), 
        'splitter': Choice(['best', 'random']),
        'min_samples_leaf': RandInt(2, 5), 
        'min_samples_split': RandInt(1, 3) 
    })
).set_hyperparams(HyperparameterSamples({
    'criterion': 'gini', 
    'splitter': 'best',
    'min_samples_leaf': 3, 
    'min_samples_split': 3 
}))